In [1]:
import gym
import minerl

C:\Users\georg\anaconda3\envs\minerl\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
%run parse_missions

In [3]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
env = gym.make("MineRLNavigate-v0", xml="my_mission_xmls/navigationFixedLTH_position.xml")

INFO:minerl.env.malmo.instance.db8287:Starting Minecraft process: ['C:\\Users\\georg\\AppData\\Local\\Temp\\tmpehfbyddg\\Minecraft\\launchClient.bat', '-port', '11336', '-env', '-runDir', 'C:\\Users\\georg\\AppData\\Local\\Temp\\tmpehfbyddg\\Minecraft\\run']
INFO:minerl.env.malmo.instance.db8287:Starting process watcher for process 13736 @ localhost:11336
DEBUG:minerl.env.malmo.instance.db8287:This mapping 'snapshot_20161220' was designed for MC 1.11! Use at your own peril.
DEBUG:minerl.env.malmo.instance.db8287:#################################################
DEBUG:minerl.env.malmo.instance.db8287:         ForgeGradle 2.2-SNAPSHOT-3966cea        
DEBUG:minerl.env.malmo.instance.db8287:  https://github.com/MinecraftForge/ForgeGradle  
DEBUG:minerl.env.malmo.instance.db8287:#################################################
DEBUG:minerl.env.malmo.instance.db8287:               Powered by MCP unknown               
DEBUG:minerl.env.malmo.instance.db8287:             http://modcoderpack.c

DEBUG:minerl.env.malmo.instance.db8287:[16:24:00] [main/WARN]: The coremod com.microsoft.Malmo.OverclockingPlugin does not have a MCVersion annotation, it may cause issues with this version of Minecraft
DEBUG:minerl.env.malmo.instance.db8287:2020-08-19 16:24:00,688 WARN Unable to instantiate org.fusesource.jansi.WindowsAnsiOutputStream
DEBUG:minerl.env.malmo.instance.db8287:2020-08-19 16:24:00,745 WARN Unable to instantiate org.fusesource.jansi.WindowsAnsiOutputStream
DEBUG:minerl.env.malmo.instance.db8287:[16:24:00] [main/INFO]: SpongePowered MIXIN Subsystem Version=0.7.5 Source=file:/C:/Users/georg/.gradle/caches/modules-2/files-2.1/org.spongepowered/mixin/0.7.5-SNAPSHOT/c989878008c2c5ff1a7c6491c4103d8faa221d19/mixin-0.7.5-SNAPSHOT.jar Service=LaunchWrapper Env=CLIENT
DEBUG:minerl.env.malmo.instance.db8287:[16:24:01] [main/INFO]: Compatibility level set to JAVA_8
DEBUG:minerl.env.malmo.instance.db8287:[16:24:01] [main/INFO]: Calling tweak class net.minecraftforge.gradle.tweakers.Core

In [ ]:
# set the environment to allow interactive connections on port 6666
# and slow the tick speed to 6666.
# env.make_interactive(port=6666, realtime=True)

In [ ]:
obs = env.reset()

In [7]:
done = False

while not done:
    action = env.action_space.sample()
    obs, reward, done, _ = env.step(action)

DEBUG:minerl.env.malmo.instance.411721:[13:26:02] [Server thread/INFO]: [STDOUT]: Warning: managed 5/20 ticks this second.
DEBUG:minerl.env.malmo.instance.411721:[13:26:02] [Client thread/INFO]: [CHAT] §l299...
DEBUG:minerl.env.malmo.instance.411721:[13:26:03] [Client thread/INFO]: [CHAT] §l298...
DEBUG:minerl.env.malmo.instance.411721:[13:26:03] [Client thread/INFO]: [CHAT] §l297...
DEBUG:minerl.env.malmo.instance.411721:[13:26:04] [Client thread/INFO]: [CHAT] §l296...
DEBUG:minerl.env.malmo.instance.411721:[13:26:04] [Client thread/INFO]: [CHAT] §l295...
DEBUG:minerl.env.malmo.instance.411721:[13:26:05] [Client thread/INFO]: [CHAT] §l294...
DEBUG:minerl.env.malmo.instance.411721:[13:26:05] [Client thread/INFO]: [CHAT] §l293...
DEBUG:minerl.env.malmo.instance.411721:[13:26:06] [Client thread/INFO]: [CHAT] §l292...
DEBUG:minerl.env.malmo.instance.411721:[13:26:06] [Client thread/INFO]: [CHAT] §l291...
DEBUG:minerl.env.malmo.instance.411721:[13:26:06] [Client thread/INFO]: [CHAT] §l290.

DEBUG:minerl.env.malmo.instance.411721:[13:26:38] [Client thread/INFO]: [CHAT] §l207...
DEBUG:minerl.env.malmo.instance.411721:[13:26:38] [Client thread/INFO]: [CHAT] §l206...
DEBUG:minerl.env.malmo.instance.411721:[13:26:39] [Client thread/INFO]: [CHAT] §l205...
DEBUG:minerl.env.malmo.instance.411721:[13:26:39] [Client thread/INFO]: [CHAT] §l204...
DEBUG:minerl.env.malmo.instance.411721:[13:26:39] [Client thread/INFO]: [CHAT] §l203...
DEBUG:minerl.env.malmo.instance.411721:[13:26:40] [Client thread/INFO]: [CHAT] §l202...
DEBUG:minerl.env.malmo.instance.411721:[13:26:40] [Client thread/INFO]: [CHAT] §l201...
DEBUG:minerl.env.malmo.instance.411721:[13:26:41] [Client thread/INFO]: [CHAT] §l200...
DEBUG:minerl.env.malmo.instance.411721:[13:26:41] [Client thread/INFO]: [CHAT] §l199...
DEBUG:minerl.env.malmo.instance.411721:[13:26:41] [Client thread/INFO]: [CHAT] §l198...
DEBUG:minerl.env.malmo.instance.411721:[13:26:42] [Client thread/INFO]: [CHAT] §l197...
DEBUG:minerl.env.malmo.instance.

DEBUG:minerl.env.malmo.instance.411721:[13:27:13] [Client thread/INFO]: [CHAT] §l113...
DEBUG:minerl.env.malmo.instance.411721:[13:27:14] [Client thread/INFO]: [CHAT] §l112...
DEBUG:minerl.env.malmo.instance.411721:[13:27:14] [Client thread/INFO]: [CHAT] §l111...
DEBUG:minerl.env.malmo.instance.411721:[13:27:14] [Client thread/INFO]: [CHAT] §l110...
DEBUG:minerl.env.malmo.instance.411721:[13:27:15] [Client thread/INFO]: [CHAT] §l109...
DEBUG:minerl.env.malmo.instance.411721:[13:27:15] [Client thread/INFO]: [CHAT] §l108...
DEBUG:minerl.env.malmo.instance.411721:[13:27:15] [Client thread/INFO]: [CHAT] §l107...
DEBUG:minerl.env.malmo.instance.411721:[13:27:16] [Client thread/INFO]: [CHAT] §l106...
DEBUG:minerl.env.malmo.instance.411721:[13:27:16] [Client thread/INFO]: [CHAT] §l105...
DEBUG:minerl.env.malmo.instance.411721:[13:27:16] [Client thread/INFO]: [CHAT] §l104...
DEBUG:minerl.env.malmo.instance.411721:[13:27:17] [Client thread/INFO]: [CHAT] §l103...
DEBUG:minerl.env.malmo.instance.

DEBUG:minerl.env.malmo.instance.411721:[13:27:46] [Client thread/INFO]: [CHAT] §l18...
DEBUG:minerl.env.malmo.instance.411721:[13:27:47] [Client thread/INFO]: [CHAT] §l17...
DEBUG:minerl.env.malmo.instance.411721:[13:27:47] [Client thread/INFO]: [CHAT] §l16...
DEBUG:minerl.env.malmo.instance.411721:[13:27:48] [Client thread/INFO]: [CHAT] §l15...
DEBUG:minerl.env.malmo.instance.411721:[13:27:48] [Client thread/INFO]: [CHAT] §l14...
DEBUG:minerl.env.malmo.instance.411721:[13:27:48] [Client thread/INFO]: [CHAT] §l13...
DEBUG:minerl.env.malmo.instance.411721:[13:27:49] [Client thread/INFO]: [CHAT] §l12...
DEBUG:minerl.env.malmo.instance.411721:[13:27:49] [Client thread/INFO]: [CHAT] §l11...
DEBUG:minerl.env.malmo.instance.411721:[13:27:49] [Client thread/INFO]: [CHAT] §l10...
DEBUG:minerl.env.malmo.instance.411721:[13:27:50] [Client thread/INFO]: [CHAT] §l9...
DEBUG:minerl.env.malmo.instance.411721:[13:27:50] [Client thread/INFO]: [CHAT] §l8...
DEBUG:minerl.env.malmo.instance.411721:[13:27

In [8]:
env.close()

INFO:minerl.env.malmo.instance.411721:[13:28:12] [EnvServerSocketHandler/INFO]: [STDOUT]: [LOGTOPY] Want to quit on disconnect.


Failed to delete the temporary minecraft directory.
